In [1]:
!pip install -q -U transformers bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.8 MB/s eta 0:00:00


In [2]:
from huggingface_hub import login
from google.colab import userdata

login(token = userdata.get("HF_TOKEN"))

In [45]:

from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("sai-santhosh/text-2-sql-Llama-3.2-3B")
model = AutoModelForCausalLM.from_pretrained("sai-santhosh/text-2-sql-Llama-3.2-3B")

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/928 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [78]:
from transformers import pipeline
import re

def get_sql_query(model,tokenizer,question,context):

    pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer)

    system_prompt = """Provide the SQL query to the question based on the context in below format
                                                    1. SQL Query: start with ```sql
                                                    2. Explanation of the query: start with ```explanation

            make the explanation clear and detail in less than 75 words"""

    prompt = pipe.tokenizer.apply_chat_template(
        [   {"role": "system", "content" : system_prompt},
            {"role": "user", "content": f"Question: {question} Context: {context}"}],
        tokenize=False,
        add_generation_prompt=True,
    )


    outputs = pipe(
        prompt,
        max_length=350,
        clean_up_tokenization_spaces=True
    )

    generated_text = outputs[0]['generated_text']

    generated_text = generated_text.replace(system_prompt,"")
    sql_query_pattern = r"```sql\n(.*?)\n```"
    sql_query_match = re.search(sql_query_pattern, generated_text, re.DOTALL)
    sql_query = sql_query_match.group(1).strip() if sql_query_match else None

    explanation_pattern = r"\*\*Explanation\:\*\*(.*)"
    explanation_match = re.search(explanation_pattern, generated_text, re.DOTALL)
    explanation = explanation_match.group(1).strip() if explanation_match else None

    print("\n\nQuery:\n-----------\n",sql_query)
    print("\n\nExplanation:\n-----------\n",explanation)

In [79]:
# SELECT name FROM Aircraft WHERE distance > (SELECT AVG(distance) FROM Aircraft)  --->  right answer
question = "Show names for all aircrafts with distances more than the average."
context  = "CREATE TABLE Aircraft (name VARCHAR, distance INTEGER)"

In [80]:
get_sql_query(model,tokenizer,question,context)

Device set to use cuda:0




Query:
-----------
 SELECT name 
FROM Aircraft 
WHERE distance > (SELECT AVG(distance) FROM Aircraft);


Explanation:
-----------
 This SQL query calculates the average distance of all aircraft in the `Aircraft` table and then selects the names of aircraft that have a distance greater than this average. This is done using a subquery to calculate the average distance, and then a regular query to select the names of aircraft that meet this condition.


In [81]:
question_1 = "Get the names of customers who spent more than the average amount."
context_1 = "CREATE TABLE Customers (name VARCHAR, spent INTEGER)"

get_sql_query(model,tokenizer,question_1,context_1)

Device set to use cuda:0




Query:
-----------
 SELECT name 
FROM Customers 
WHERE spent > (SELECT AVG(spent) FROM Customers);


Explanation:
-----------
 This SQL query calculates the average amount spent by all customers and then selects the names of customers who have spent more than this average amount. It uses a subquery to calculate the average spent and then compares each customer's spent amount to this average in the main query.


In [82]:
question_2 = "Find the highest salary from the Employees table."
context_2 = "CREATE TABLE Employees (name VARCHAR, salary INTEGER)"

get_sql_query(model,tokenizer,question_2,context_2)

Device set to use cuda:0




Query:
-----------
 SELECT MAX(salary) AS highest_salary
FROM Employees;


Explanation:
-----------
 This SQL query uses the `MAX` aggregation function to find the highest salary from the `salary` column in the `Employees` table. The result is given an alias `highest_salary` for clarity. This query assumes that the `salary` column contains only numeric values. If there are multiple employees with the same highest salary, this query will return one of them.


In [83]:
question_3 = "Show all orders that were placed by customers from New York."
context_3 = "CREATE TABLE Orders (order_id INTEGER, customer_id INTEGER, city VARCHAR)"

get_sql_query(model,tokenizer,question_3,context_3)

Device set to use cuda:0




Query:
-----------
 SELECT *
FROM Orders
WHERE city = 'New York';


Explanation:
-----------
 This SQL query retrieves all rows from the `Orders` table where the `city` column is 'New York'. It uses a simple `WHERE` clause to filter the results based on the specified condition. The `SELECT *` statement selects all columns (`order_id`, `customer_id`, and `city`) from the table.


In [84]:
question_4 = "Find the names of customers who have placed orders with a total amount greater than the average order amount."
context_4 = "CREATE TABLE Customers (customer_id INTEGER, name VARCHAR); CREATE TABLE Orders (order_id INTEGER, customer_id INTEGER, amount INTEGER);"

get_sql_query(model,tokenizer,question_4,context_4)

Device set to use cuda:0




Query:
-----------
 SELECT C.name 
FROM Customers C 
JOIN (
  SELECT customer_id, AVG(amount) as avg_amount 
  FROM Orders 
  GROUP BY customer_id
) AS subquery 
ON C.customer_id = subquery.customer_id 
WHERE subquery.avg_amount < (SELECT AVG(amount) FROM Orders);


Explanation:
-----------
 This SQL query joins the Customers table with a subquery that calculates the average order amount for each customer. It then selects the names of customers who have an average order amount greater than the overall average order amount. The subquery uses the `AVG` function with `GROUP BY` to calculate the average order amount for each customer, and the outer query joins this result with the Customers table to retrieve the customer names.


In [85]:
question_5 = "Get the names of products that were ordered by customers in New York who spent more than the average amount."
context_5 = "CREATE TABLE Customers (customer_id INTEGER, name VARCHAR, city VARCHAR); CREATE TABLE Orders (order_id INTEGER, customer_id INTEGER, amount INTEGER); CREATE TABLE Products (product_id INTEGER, name VARCHAR); CREATE TABLE Order_Items (order_id INTEGER, product_id INTEGER);"

get_sql_query(model,tokenizer,question_5,context_5)

Device set to use cuda:0




Query:
-----------
 SELECT DISTINCT p.name 
FROM Customers c 
JOIN Orders o ON c.customer_id = o.customer_id 
JOIN Order_Items oi ON o.order_id = oi.order_id 
JOIN Products p ON oi.product_id = p.product_id 
WHERE c.city = 'New York' 
AND o.amount > (SELECT AVG(amount) FROM Orders);


Explanation:
-----------
 This SQL query joins the `Customers`, `Orders`, `Order_Items`, and `Products` tables to retrieve the names of products ordered by customers in New York who spent more than the average amount. It first calculates the average amount spent by all customers using a subquery, then filters the results to only include customers in New York and orders with amounts greater than the average. The `SELECT DISTINCT` statement ensures that each product name is only listed once.
